## 1. Repositório de dados:
https://github.com/cmmonteforte/Grupo9-ProjetoAplicadoIV

## 2. Base de dados:
https://covid.saude.gov.br/

## 3. Objetivo

Neste projeto, buscamos desenvolver um modelo preditivo que analise dados históricos de casos e óbitos causados pela COVID-19 no território brasileiro, tentando prever a quantidade de novos casos para que assim seja possível apoiar a tomada de decisão das autoridades na gestão de saúde pública.

Esperamos atingir esse objetivo através das seguintes metas:

* Coletar e processar dados oficiais de casos e óbitos relacionados à COVID-19 no Brasil;
* Aplicar e comparar técnicas de modeloagem de séries temporais;
* Validar o desempenho dos modelos preditivos através de métricas de erros e testes estatísticos;
* Gerar um modelo com bom desempenho, capaz de prever a quantidade de casos ou óbitos nos próximos dias, semanas ou meses.

## 4. Definição dos pacotes

In [1]:
#Importando os pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable

%matplotlib inline
sns.set_style('darkgrid')

## 5. Geração da tabela de Metadados



In [2]:
df = pd.read_csv("df_covid.csv")
df.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-07-01,26,210147125.0,18622304.0,65163,520095,2029,16858632.0,1180443.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-07-02,26,210147125.0,18687469.0,65165,521952,1857,16931272.0,1170937.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-07-03,26,210147125.0,18742025.0,54556,523587,1635,16989351.0,1176166.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-07-04,27,210147125.0,18769808.0,27783,524417,830,17033808.0,1184630.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2021-07-05,27,210147125.0,18792511.0,22703,525112,695,17082876.0,1162515.0,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10081354 entries, 0 to 10081353
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   regiao                  object 
 1   estado                  object 
 2   municipio               object 
 3   coduf                   int64  
 4   codmun                  float64
 5   codRegiaoSaude          float64
 6   nomeRegiaoSaude         object 
 7   data                    object 
 8   semanaEpi               int64  
 9   populacaoTCU2019        float64
 10  casosAcumulado          float64
 11  casosNovos              int64  
 12  obitosAcumulado         int64  
 13  obitosNovos             int64  
 14  Recuperadosnovos        float64
 15  emAcompanhamentoNovos   float64
 16  interior/metropolitana  float64
dtypes: float64(7), int64(5), object(5)
memory usage: 1.3+ GB


### 5.1 Tabela com os metadados

In [4]:
# extrair nome das colunas
nomes_colunas = df.columns.tolist()

# extrair tipos de dados
tipos_dados = df.dtypes.tolist()

# extrair número de valores nulos
num_nulos = df.isnull().sum().tolist()

# criar lista com exemplos de valores não nulos e origem dos dados
exemplos = []
for coluna in nomes_colunas:
    exemplo = df[coluna].dropna().iloc[0]  # pega o primeiro valor não nulo
    exemplos.append(exemplo)

# origem dos dados
origem = ['df_covid'] * len(nomes_colunas)

# criar a tabela PrettyTable
table = PrettyTable()

# Definir os nomes das colunas
table.field_names = ["Variável", "Tipo", "Exemplo", "% Nulos", "Origem"]

# Adicionar as linhas na tabela
for i in range(len(nomes_colunas)):
    table.add_row([nomes_colunas[i], tipos_dados[i], exemplos[i], 
                  f"{round((num_nulos[i] / df.shape[0]) * 100, 2)}%", origem[i]])

# Exibir
print(table)

+------------------------+---------+-----------------------+---------+----------+
|        Variável        |   Tipo  |        Exemplo        | % Nulos |  Origem  |
+------------------------+---------+-----------------------+---------+----------+
|         regiao         |  object |         Brasil        |   0.0%  | df_covid |
|         estado         |  object |           RO          |  0.02%  | df_covid |
|       municipio        |  object | Alta Floresta D'Oeste |  0.88%  | df_covid |
|         coduf          |  int64  |           76          |   0.0%  | df_covid |
|         codmun         | float64 |        110000.0       |  0.51%  | df_covid |
|     codRegiaoSaude     | float64 |        11005.0        |  0.88%  | df_covid |
|    nomeRegiaoSaude     |  object |      ZONA DA MATA     |  0.88%  | df_covid |
|          data          |  object |       2021-07-01      |   0.0%  | df_covid |
|       semanaEpi        |  int64  |           26          |   0.0%  | df_covid |
|    populacaoTC